## ScanDatabase builder for fast query

Build a comprehensive, fast query-able parque 'scans database'

see API documentation for more details

In [ ]:
from datetime import date
from geecs_data_utils import ScanData
from geecs_data_utils.scans_database.builder import ScanDatabaseBuilder

# Inputs
data_root = ScanData.paths_config.base_path
experiment = "Undulator"
output_path = data_root / experiment / "scan_database_parquet"
date_range = (date(2022, 1, 3), date(2025, 8, 5))

ScanDatabaseBuilder.stream_to_parquet(
    data_root=data_root,
    experiment=experiment,
    output_path=output_path,
    date_range=date_range,
    buffer_size=50,
    max_scans=1000000,
    mode="overwrite",  # or 'append' or 'overwrite'
)

 After database is initially built, new scan entries are easily added using mode='append' and data_range = None

In [ ]:
# Picks up from the last date in _update_log.json through today
ScanDatabaseBuilder.stream_to_parquet(
    data_root=data_root,
    experiment=experiment,
    output_path=output_path,
    date_range=None,  # <- auto-resume via sidecar
    buffer_size=50,
    mode="append",
)

Filter scans database based on many flexible criteria

In [10]:
from geecs_data_utils.scans_database.database import ScanDatabase
from datetime import date

data_root = ScanData.paths_config.base_path
experiment = "Undulator"
output_path = data_root / experiment / "scan_database_parquet"

# initialize ScanDabase object
db = ScanDatabase(output_path)

# apply filters

# Filter by date first, for fast initial filter
db.date_range(date(2025, 8, 5), date(2025, 8, 8))

# can filter by scanparameter, case insensitive. use alias, var name or any part
db.filter_scan_parameter_contains("shotnumber")

# Filter by ecs live dump entries: (device_like, variable_like, target value, tolerance)
db.filter_ecs_value_within("hexapod", "y", target=18.5, tol=0.5)

# use defined named filters autoloaded based on experiment name. Can have various date range validities
db.apply("PMQ_inserted")

# convert result to dataframe
df = db.to_df()

print("rows:", len(df))
df.head(5)

[INFO] Loaded 1/1 filters from undulator.yml
rows: 10


,day,number,experiment,scalar_data_file,tdms_file,non_scalar_devices,scan_parameter,start,end,step_size,...,scan_mode,scan_description,background,scan_metadata_raw_fields,ecs_dump,has_analysis_dir,notes,year,month,__ecs
0,7,1,Undulator,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,[Z_Test_Scope],Shotnumber,0,1,1,...,noscan,. scanning Shotnumber. None,0,"{""Scan No"": ""1"", ""ScanStartInfo"": "". scanning ...","{""experiment_name"": ""Undulator"", ""devices"": [{...",1,<NA>,2025,8,"{'experiment_name': 'Undulator', 'devices': [{..."
1,7,2,Undulator,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,"[Z_Test_Scope, Z_Test_Scope_2]",Shotnumber,0,1,1,...,noscan,. scanning Shotnumber. None,0,"{""Scan No"": ""2"", ""ScanStartInfo"": "". scanning ...","{""experiment_name"": ""Undulator"", ""devices"": [{...",1,<NA>,2025,8,"{'experiment_name': 'Undulator', 'devices': [{..."
2,7,3,Undulator,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,"[Z_Test_Scope, Z_Test_Scope_2]",Shotnumber,0,1,1,...,noscan,. scanning Shotnumber. None,0,"{""Scan No"": ""3"", ""ScanStartInfo"": "". scanning ...","{""experiment_name"": ""Undulator"", ""devices"": [{...",1,<NA>,2025,8,"{'experiment_name': 'Undulator', 'devices': [{..."
3,7,4,Undulator,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,"[Z_Test_Scope, Z_Test_Scope_2]",Shotnumber,0,1,1,...,noscan,. scanning Shotnumber. None,0,"{""Scan No"": ""4"", ""ScanStartInfo"": "". scanning ...","{""experiment_name"": ""Undulator"", ""devices"": [{...",1,<NA>,2025,8,"{'experiment_name': 'Undulator', 'devices': [{..."
4,7,5,Undulator,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,Z:\data\Undulator\Y2025\08-Aug\25_0807\scans\S...,"[UC_ALineEBeam3, U_BCaveICT, Z_Test_Scope, Z_T...",Shotnumber,0,1,1,...,noscan,statistics scan on BAM. scanning Shotnumber. None,0,"{""Scan No"": ""5"", ""ScanStartInfo"": ""statistics ...","{""experiment_name"": ""Undulator"", ""devices"": [{...",1,<NA>,2025,8,"{'experiment_name': 'Undulator', 'devices': [{..."
